## IMPORTING LIBRARIES

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## IMPORTING DATASETS

In [53]:
sample_df = pd.read_csv('Sample.csv')
ground_truth_df = pd.read_csv('ground-truth.csv')

In [3]:
#sample_df.head()

## Setting up the LLaVa model from TERMINAL
LLaVA (Large Language-and-Vision Assistant) is an end-to-end trained large multimodal model that connects a vision encoder and LLM for general-purpose visual and language understanding.

The LLaVA model exhibits remarkable interactive capabilities between images and videos

In [4]:
#ground_truth_df.head()

In [45]:
#Prompt Engineering is important for the response from the LLM
prompt = """You are an assistant tasked to watch videos. You will be given description, speech, transcription etc for each video.
                              You will be asked a question related to the video. You only have to answer as Yes or No
                              Here is the description: As a man speeds down a country road in his Audi Q7, its pre-sense safety technology responds to a deer leaping suddenly in front of his car. Without missing a beat, the vehicle skids to a stop before the hair on the startled man's arms has time to rise. The Audi pre-sense is now available in the Audi A4.

                              Here is the speech:  Audi presens can help prepare for and in some cases help prevent collisions faster than well, you progress is amplifying your instincts. Audi Precent found in the Audi A four, get an exceptional offer at your local Audi dealer.
                              You will be asked the following three questions
                              1. Is there a call to go online?
                              Guidelines to answer this question: Say yes only if a website is shown at the end of the video or it is present in description or speech. Else say no
                              2. Is there mention of something free?
                              Guidelines to answer this question: Say yes only if the word free is explicitly mentioned in speech or description or if you see the word free in the video. Else say no.
                              3. Does the ad have cute elements?
                              Guidelines to answer this question: Say yes only if something is animated or small animals or babies. Else say no.

                              Here is your question: Is there mention of something free?"""

In [46]:
url = "http://127.0.0.1:5000/generate"
headers = {
    "Content-Type": "application/json"
}
data = {
    "file_paths": ["sample/1667694.mp4"],
    #"text": "Is there an incentive to buy?"
    "text": prompt
}

response = requests.post(url, headers=headers, data=json.dumps(data))

print(response.text)


{
  "response": "No</s>"
}



In [7]:
# Code for creating a prompt and answering all questions from the questionnaire for each video

In [19]:
video_folder_path = 'sample/'

In [10]:
# Iterate over each file in the directory
for filename in os.listdir(video_folder_path):
    if filename.endswith(".mp4"):
        video_id = filename.split('.')[0]  # Assuming the filename is the creative_data_id

        video_data = sample_df[sample_df['creative_data_id'].astype(str) == video_id]

        if not video_data.empty:
            description = video_data['creative_data_description'].iloc[0]
            speech = video_data['speech'].iloc[0]
            prompt = f"""You are an assistant tasked to watch videos. You will be given description, speech, transcription etc for each video.
            You will be asked a question related to the video. You only have to answer as Yes or No

            Here is the description: {description}
            \n
            Here is the speech:  {speech}"""

            print("Processing video:", filename)
            print("Prompt:", prompt)
            print("\n")
        else:
            print("No data found for video ID:", video_id)

# Handle the case if no videos are found
if not any(filename.endswith(".mp4") for filename in os.listdir(video_folder_path)):
    print("No video files found in the directory.")

Processing video: 1526213.mp4
Prompt: You are an assistant tasked to watch videos. You will be given description, speech, transcription etc for each video.
You will be asked a question related to the video. You only have to answer as Yes or No

Here is the description: As a man speeds down a country road in his Audi Q7, its pre-sense safety technology responds to a deer leaping suddenly in front of his car. Without missing a beat, the vehicle skids to a stop before the hair on the startled man's arms has time to rise. The Audi pre-sense is now available in the Audi A4.


Here is the speech:  Audi presens can help prepare for and in some cases help prevent collisions faster than well, you progress is amplifying your instincts. Audi Precent found in the Audi A four, get an exceptional offer at your local Audi dealer.


Processing video: 1471363.mp4
Prompt: You are an assistant tasked to watch videos. You will be given description, speech, transcription etc for each video.
You will be ask

In [4]:
# List of questions for the questionnaire (adding guidelines to make the response more accurate)
questions = [
    "Is there a call to go online? Guidelines: Say yes only if a website is shown at the end of the video or it is mentioned in the description or speech. Else say no.",
    "Is online contact information provided? Guidelines: Say yes only if any form of online contact information like an email, website URL, or social media link is shown or mentioned. Else say no.",
    "Is there a visual or verbal call to purchase? Guidelines: Say yes if the ad directly encourages viewers to buy a product or service, such as through phrases like 'buy now' or 'order today'. Else say no.",
    "Does the ad portray a sense of urgency to act? Guidelines: Say yes if there's a prompt suggesting limited time, like 'offer ends soon' or 'hurry, while supplies last'. Else say no.",
    "Is there an incentive to buy? Guidelines: Say yes if the ad offers discounts, coupons, sales, or exclusive deals. Else say no.",
    "Is offline contact information provided? Guidelines: Say yes if there's mention of physical contact details like a phone number or store location. Else say no.",
    "Is there mention of something free? Guidelines: Say yes only if the word 'free' is explicitly mentioned in the speech or description, or if you see the word free in the video. Else say no.",
    "Does the ad mention at least one specific product or service? Guidelines: Say yes if specific products or services are detailed. Else say no.",
    "Is there any verbal or visual mention of the price? Guidelines: Say yes if the price of a product or service is disclosed or shown. Else say no.",
    "Does the ad show the brand multiple times? Guidelines: Say yes if the brand's logo or name appears several times throughout the ad. Else say no.",
    "Does the ad show the brand exactly once at the end of the ad? Guidelines: Say yes if the brand's logo or name is shown only once, and it is at the end. Else say no.",
    "Is the ad intended to affect the viewer emotionally? Guidelines: Say yes if the ad aims to elicit emotions, whether positive or negative. Else say no.",
    "Does the ad give you a positive feeling about the brand? Guidelines: Say yes if the ad overall leaves a positive impression of the brand, regardless of personal opinion. Else say no.",
    "Does the ad have a story arc? Guidelines: Say yes if the ad has a clear beginning, middle, and end that tells a story. Else say no.",
    "Does the ad have a reversal of fortune? Guidelines: Say yes if the ad shows a significant change in circumstances, for better or worse. Else say no.",
    "Does the ad have relatable characters? Guidelines: Say yes if the characters in the ad are relatable or evoke empathy. Else say no.",
    "Is the ad creative/cleve? Guidelines: Say yes if the ad demonstrates originality or cleverness. Else say no.",
    "Is the ad intended to be funny? Guidelines: Say yes if the ad aims to be humorous, regardless of personal opinion on the humor. Else say no.",
    "Does this ad provide sensory stimulation? Guidelines: Say yes if the ad includes appealing visuals, enticing sounds, or other sensory details that capture attention. Else say no.",
    "Is the ad visually pleasing? Guidelines: Say yes if the ad is aesthetically attractive or has high-quality graphics and imagery. Else say no.",
    "Does the ad have cute elements? Guidelines: Say yes only if the ad contains animated features, small animals, or babies. Else say no."
]

In [5]:
results_df = pd.DataFrame(columns=['creative_data_id'] + [f'question_{i+1}' for i in range(len(questions))])

In [61]:
#wrote a script with flask to serve api and make it easier
# API details
url = "http://127.0.0.1:5000/generate"
headers = {"Content-Type": "application/json"}

# Iterating over each file in the directory
for filename in os.listdir(video_folder_path):
    if filename.endswith(".mp4"):
        video_id = filename.split('.')[0]  # Assuming the filename is the creative_data_id

        video_data = sample_df[sample_df['creative_data_id'].astype(str) == video_id]

        if not video_data.empty:
            description = video_data['creative_data_description'].iloc[0]
            speech = video_data['speech'].iloc[0]
            results = [video_id]

            for question in questions:
                prompt = f"""You are an assistant tasked to watch videos. You will be given description and speech for each video.
                              You will be asked a question related to the video. You only have to answer as Yes or No
                              Here is the description: {description}
                              Here is the speech:  {speech}
                              Here is your question: {question}"""

                data = json.dumps({"file_paths": [f"sample/{video_id}.mp4"], "text": prompt})
                response = requests.post(url, headers=headers, data=data)
                answer = json.loads(response.text)['response'].split("</s>")[0]
                results.append('yes' if answer.lower() == 'yes' else 'no')

            # Append the results for this video to the DataFrame
            print("Processing video:", filename)
            print(results)
            results_df.loc[len(results_df)] = results

# Save the DataFrame to a CSV file
results_df.to_csv('results.csv', index=False)
print("Completed processing all videos.")


Streaming output truncated to the last 5000 lines.

























Processing video: 3170124.mp4
['3170124', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']










































Processing video: 1958838.mp4
['1958838', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']










































Processing video: 1683011.mp4
['1683011', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']










































Processing video: 2454123.mp4
['2454123', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']










































Pro

In [6]:
#results_df

## EVALUATION

ground_truth_df needs to be analyzed and pre processed to calculate final majority vote and to make it more easy to compare with the results_df

In [1]:
#ground_truth_df.groupby('creative_data_id').first()

In [2]:
#ground_truth_df.isnull().sum()

In [5]:
#ground_truth_df.columns

In [69]:
#Dropping null value columns and columns that are not useful for comparing with the result dataframe
#ground_truth_df.drop(['Timestamp','Any additional feedback or things we should be aware of? ','Please enter the video identifier one more time (e.g. 123456789.mp4)', 'If "yes" to the above, which of the following emotions is closest to the emotion that the ad was intending the viewer to feel? (Select all that apply.)', 'If yes to the above, did the ad successfully affect you emotionally, as intended?', 'If yes to the above, was the ad successfully funny, as intended?' , 'If yes to the above, write the name of the famous person, if known.', 'What was the slogan presented in the ad, if any?', 'Was there a famous person in this ad? ',
      #  'What happened in this ad? (Answer in 2-3 sentences each)',
      #  "What was/were the company's goal(s) with this ad? Choose (potentially multiple) from:",
      #  'How successful was the ad in achieving its goal(s)?',
      #  'How much did you like the ad? (1. Strongly dislike, 2. Dislike, 3. Neither Like or Dislike, 4. Like, 5. Strongly Like)',
      #  'After addressing the specific survey items, write a general description of the ad. You can use answers to the questions above to formulate your answer. Your description should include:\nBrand and Product Identification: \nSpecify the brand and whether a product is being advertised. (1 sentence)\nVisual Elements: Describe what is seen on the screen, including setting, characters, and any text or graphics. (max 2 sentences)\nAuditory Elements: Note what is heard, such as dialogue, voice-over, music, or sound effects. (max 2 sentences)\n' ], axis=1, inplace=True)

In [70]:
# Function to calculate the majority vote for each question within each video
def calculate_majority_vote(group):
    results = {}
    for col in group.columns:
        if col not in ['creative_data_id']:
            counts = group[col].value_counts()
            if len(counts) > 1 and counts.iloc[0] == counts.iloc[1]:
                results[col] = group[col].mode()[0]
            else:
                results[col] = counts.idxmax()
    return pd.Series(results)

In [71]:
#grouping by 'creative_data_id' and calculating majority vote
majority_votes_df = ground_truth_df.groupby('creative_data_id').apply(calculate_majority_vote)
majority_votes_df = majority_votes_df.reset_index()

# Save the processed data to a CSV file
majority_votes_df.to_csv('majority_votes.csv', index=False)
print("Processed majority votes saved to 'majority_votes.csv'.")

Processed majority votes saved to 'majority_votes.csv'.


In [72]:
results_df.sort_values('creative_data_id', inplace=True)
majority_votes_df.sort_values('creative_data_id', inplace=True)

In [73]:
#Need to rename the column names of ground-truth to compare with results_df
original_columns = [
    'creative_data_id',
    'Is there a call to go online (e.g., shop online, visit the Web)? ',
    'Is there online contact information provided (e.g., URL, website)? ',
    'Is there a visual or verbal call to purchase (e.g., buy now, order now)?',
    'Does the ad portray a sense of urgency to act (e.g., buy before sales ends, order before ends)? ',
    'Is there an incentive to buy (e.g., a discount, a coupon, a sale or "limited time offer")? ',
    'Is there offline contact information provided (e.g., phone, mail, store location)?',
    'Is there mention of something free? ',
    'Does the ad mention at least one specific product or service (e.g., model, type, item)? ',
    'Is there any verbal or visual mention of the price?',
    'Does the ad show the brand (logo, brand name) or trademark (something that most people know is the brand) multiple times?\n\nFor example, Nike ads often have the "swoosh" logo prominently displayed on shoes and apparel worn by celebrity athletes. The "Just Do It" slogan is another Nike trademark frequently included.',
    'Does the ad show the brand or trademark exactly once at the end of the ad?',
    'Is the ad intended to affect the viewer emotionally, either with positive emotion (fun, joy), negative emotion (sad, anxious) or another type of emotion? (Note: You may not personally agree, but assess if that was the intention.)',
    'Does the ad give you a positive feeling about the brand? ',
    'Does the ad have a story arc, with a beginning and an end? ',
    'Does the ad have a reversal of fortune, where something changes for the better, or changes for the worse?',
    'Does the ad have relatable characters? ', 'Is the ad creative/clever?',
    'Is the ad intended to be funny? (Note: You may not personally agree, but assess if that was the intention.) ',
    'Does this ad provide sensory stimulation (e.g., cool visuals, arousing music, mouth-watering)? ',
    'Is the ad visually pleasing?',
    'Does the ad have cute elements like animals, babies, animated, characters, etc?'
]

new_columns = {original_columns[i]: f'question_{i}' for i in range(1, len(original_columns))}
majority_votes_df.rename(columns=new_columns, inplace=True)

print("Columns have been renamed.")

Columns have been renamed.


In [74]:
results_df['creative_data_id'] = majority_votes_df['creative_data_id']

In [75]:
for col in majority_votes_df.columns:
    if 'question' in col:
        majority_votes_df[col] = majority_votes_df[col].str.lower()

# Metrics calculation

In [79]:
for i in range(1, 22):
    question_col = f'question_{i}'
    # Converting yes/no to 1/0
    y_true = (majority_votes_df[question_col] == 'yes').astype(int)
    y_pred = (results_df[question_col] == 'yes').astype(int)

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    print(f"Metrics for {question_col}:")
    print(f"  Agreement Percentage: {accuracy * 100:.2f}%")
    print(f"  F1 Score: {f1:.2f}")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall: {recall:.2f}")
    print()

Metrics for question_1:
  Agreement Percentage: 36.00%
  F1 Score: 0.44
  Precision: 0.29
  Recall: 0.97

Metrics for question_2:
  Agreement Percentage: 45.33%
  F1 Score: 0.61
  Precision: 0.45
  Recall: 0.98

Metrics for question_3:
  Agreement Percentage: 40.00%
  F1 Score: 0.54
  Precision: 0.37
  Recall: 0.98

Metrics for question_4:
  Agreement Percentage: 26.00%
  F1 Score: 0.40
  Precision: 0.25
  Recall: 1.00

Metrics for question_5:
  Agreement Percentage: 42.67%
  F1 Score: 0.59
  Precision: 0.42
  Recall: 1.00

Metrics for question_6:
  Agreement Percentage: 20.00%
  F1 Score: 0.29
  Precision: 0.17
  Recall: 1.00

Metrics for question_7:
  Agreement Percentage: 20.00%
  F1 Score: 0.15
  Precision: 0.08
  Recall: 1.00

Metrics for question_8:
  Agreement Percentage: 83.33%
  F1 Score: 0.91
  Precision: 0.85
  Recall: 0.98

Metrics for question_9:
  Agreement Percentage: 10.67%
  F1 Score: 0.00
  Precision: 0.00
  Recall: 0.00

Metrics for question_10:
  Agreement Percentag

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
